### Assignment 4
Josh Gilheany 22221584


First, I accessed the NCBI page for the HFE gene through python.

In [8]:
from IPython.display import HTML
HTML('<iframe src=http://www.ncbi.nlm.nih.gov/nuccore/NG_008720.2 width=1000 height=350></iframe>')

***

Then I used entrez to pull down the data for the HFE gene, double-checking that it was a seqrecord object.

In [11]:
from Bio import Entrez
from Bio import SeqIO

try:
    Entrez.email = "John.Genomic@example.com"   #Note: John Genomic is not a real person, he is a fabrication
    handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="NG_008720.2")
    hfe = SeqIO.read(handle, "gb")
    handle.close()
    print(type(hfe))
except:
    print('Failed to fetch record. :[')
    
#we have HFE from genbank, now the ~fun~ begins

<class 'Bio.SeqRecord.SeqRecord'>


In [1]:
# load up our libraries
from Bio import SeqIO
from Bio import Entrez

Entrez.email = 'appropriate.address@provider.com' # replace with appropriate email address
handle = Entrez.efetch(db="nucleotide", rettype='gb', retmode='text', id='NG_008720')
with open('NG_008720.gb','w') as outfile:
    outfile.write(handle.read())   
    
handle = Entrez.efetch(db="nucleotide", rettype='gb', retmode='text', id='NG_008720')

for rec in SeqIO.parse(handle,'genbank'):
    if rec.features:
        for feature in rec.features:
            if feature.type == "CDS":
                coding_seq = feature.location.extract(rec).seq
                protein_seq = feature.qualifiers["translation"]
                pass

print(protein_seq)

['MGPRARPALLLLMLLQTAVLQGRLLRSHSLHYLFMGASEQDLGLSLFEALGYVDDQLFVFYDHESRRVEPRTPWVSSRISSQMWLQLSQSLKGWDHMFTVDFWTIMENHNHSKESHTLQVILGCEMQEDNSTEGYWKYGYDGQDHLEFCPDTLDWRAAEPRAWPTKLEWERHKIRARQNRAYLERDCPAQLQQLLELGRGVLDQQVPPLVKVTHHVTSSVTTLRCRALNYYPQNITMKWLKDKQPMDAKEFEPKDVLPNGDGTYQGWITLAVPPGEEQRYTCQVEHPGLDQPLIVIWEPSPSGTLVIGVISGIAVFVVILFIGILFIILRKRQGSRGAMGHYVLAERE']


In [2]:
print(coding_seq)

ATGGGCCCGCGAGCCAGGCCGGCGCTTCTCCTCCTGATGCTTTTGCAGACCGCGGTCCTGCAGGGGCGCTTGCTGCGTTCACACTCTCTGCACTACCTCTTCATGGGTGCCTCAGAGCAGGACCTTGGTCTTTCCTTGTTTGAAGCTTTGGGCTACGTGGATGACCAGCTGTTCGTGTTCTATGATCATGAGAGTCGCCGTGTGGAGCCCCGAACTCCATGGGTTTCCAGTAGAATTTCAAGCCAGATGTGGCTGCAGCTGAGTCAGAGTCTGAAAGGGTGGGATCACATGTTCACTGTTGACTTCTGGACTATTATGGAAAATCACAACCACAGCAAGGAGTCCCACACCCTGCAGGTCATCCTGGGCTGTGAAATGCAAGAAGACAACAGTACCGAGGGCTACTGGAAGTACGGGTATGATGGGCAGGACCACCTTGAATTCTGCCCTGACACACTGGATTGGAGAGCAGCAGAACCCAGGGCCTGGCCCACCAAGCTGGAGTGGGAAAGGCACAAGATTCGGGCCAGGCAGAACAGGGCCTACCTGGAGAGGGACTGCCCTGCACAGCTGCAGCAGTTGCTGGAGCTGGGGAGAGGTGTTTTGGACCAACAAGTGCCTCCTTTGGTGAAGGTGACACATCATGTGACCTCTTCAGTGACCACTCTACGGTGTCGGGCCTTGAACTACTACCCCCAGAACATCACCATGAAGTGGCTGAAGGATAAGCAGCCAATGGATGCCAAGGAGTTCGAACCTAAAGACGTATTGCCCAATGGGGATGGGACCTACCAGGGCTGGATAACCTTGGCTGTACCCCCTGGGGAAGAGCAGAGATATACGTGCCAGGTGGAGCACCCAGGCCTGGATCAGCCCCTCATTGTGATCTGGGAGCCCTCACCGTCTGGCACCCTAGTCATTGGAGTCATCAGTGGAATTGCTGTTTTTGTCGTCATCTTGTTCATTGGAATTTTGTTCATAATATTAAGGAAGAGGCAGG

***

From there I was able to convert the CDS regions of the object into a string of coding sequence by taking advantage of the ability to manipulate features of seqrecord objects, extracting the matching sequence into string called *seqstr* which would be used later.

I also got the length of the cds for verification purposes, and printed out the actual start/stop regions in the code, for later use.

In [205]:
from Bio import SeqIO

gene_sequence = []
for i in hfe.features:
    if i.type == 'CDS':
        gene_sequence.append(i.extract(hfe.seq))
    else:
        None

seqstr = gene_sequence[0]
print(len(seqstr))

cdsloc = []
for feature in hfe.features:
    if feature.type == "CDS":
        print(feature.location)
        cdsloc.append(feature.location)


1047
join{[5160:5236](+), [8560:8824](+), [9033:9309](+), [10404:10680](+), [10838:10952](+), [11905:11946](+)}


*** 
Experimentation with pulling out the actual exon regions by their nucleotide number, I wasn't able to crack this before the end of the class.

In [206]:

exonscds = []
for i in hfe.features:
    if i.type == 'exon':
        exonscds.append(i.extract(hfe.seq))
    
        
print(exonscds)

[]


***

I created a dictionary of the CDS regions by their nucleotide numbers for verification purposes, checking that the length matched what I pulled out earlier.

In [149]:
sequence = hfe.seq
cdslocs = {5160:5236, 8560:8824, 9033:9309, 10404:10680, 10838:10952, 11905:11946}
#hardcoded cds locations from previous part

coding = ""
for key in cdslocs:
    coding += sequence[key:cdslocs[key]] #extracts coding sequence

locsum = 0
for key in cdslocs:
    locsum += (cdslocs[key] - key)

print(locsum) #verify same length
print(len(coding))

#double checks that the sequence is the same as what we pulled from above

1047
1047


***
Similarly, I did not manage to crack pulling down the translated sequence from the data we had at the time and so I resorted to just comparing the translated sequence from a version I copied from the website. 

Apologies for the lack of slickness.

In [199]:
coding = seqstr

protein_sequence = coding.translate(table=11, cds=True)
print("Translated into amino acids:")
print(protein_sequence)



ncbitranslation="MGPRARPALLLLMLLQTAVLQGRLLRSHSLHYLFMGASEQDLGLSLFEALGYVDDQLFVFYDHESRRVEPRTPWVSSRISSQMWLQLSQSLKGWDHMFTVDFWTIMENHNHSKESHTLQVILGCEMQEDNSTEGYWKYGYDGQDHLEFCPDTLDWRAAEPRAWPTKLEWERHKIRARQNRAYLERDCPAQLQQLLELGRGVLDQQVPPLVKVTHHVTSSVTTLRCRALNYYPQNITMKWLKDKQPMDAKEFEPKDVLPNGDGTYQGWITLAVPPGEEQRYTCQVEHPGLDQPLIVIWEPSPSGTLVIGVISGIAVFVVILFIGILFIILRKRQGSRGAMGHYVLAERE"
#from website

protein_sequence == ncbitranslation
#They agree


Translated into amino acids:
MGPRARPALLLLMLLQTAVLQGRLLRSHSLHYLFMGASEQDLGLSLFEALGYVDDQLFVFYDHESRRVEPRTPWVSSRISSQMWLQLSQSLKGWDHMFTVDFWTIMENHNHSKESHTLQVILGCEMQEDNSTEGYWKYGYDGQDHLEFCPDTLDWRAAEPRAWPTKLEWERHKIRARQNRAYLERDCPAQLQQLLELGRGVLDQQVPPLVKVTHHVTSSVTTLRCRALNYYPQNITMKWLKDKQPMDAKEFEPKDVLPNGDGTYQGWITLAVPPGEEQRYTCQVEHPGLDQPLIVIWEPSPSGTLVIGVISGIAVFVVILFIGILFIILRKRQGSRGAMGHYVLAERE


True

***
I sliced the code at the region indicated to show if the SNP was present. It was not.

In [200]:
#4

print(f"At position 845 the reference genome has a {coding[844:845]} nucleotide")



At position 845 the reference genome has a G nucleotide


***
I used a modified version of code from earlier to see what exon the SNP would be located at.

In [202]:
#5
locsum = 0
counter = 0
for key in cdslocs:
    if locsum < 845:
        locsum += (cdslocs[key] - key)
        counter += 1
    else:
        pass
print(f"Locus found in exon {counter}")
    

#Position 845 is on the 4th exon near the end


Locus found in exon 4


***

Only RsaI demonstrated any activity on the given sequence. It would cut twice on mutated, hemochromatosis genome and once on the reference genome. In terms of an assay, DNA from a positive result would be in smaller sizes than that from a person whose test came back negative, allowing it to be used as a diagnostic tool.

In [208]:
#6

ref100seq = coding[795:895]
mut100seq = ref100seq[:49] + 'A' + ref100seq[50:]
#creation of mutant and reference 100bp windows around C282Y

from Bio.Restriction import HinfI, RsaI, EcoRI
print(HinfI.search(ref100seq))
print(HinfI.search(mut100seq))

print(RsaI.search(ref100seq))
print(RsaI.search(mut100seq))

print(EcoRI.search(ref100seq))
print(EcoRI.search(mut100seq))

[]
[]
[21]
[21, 50]
[]
[]
